# Optimizing molecular structure


The initial structure can be provided manually, or as constructed using SMILES strings (see [previous](wf_build) section). Here we will do the latter to illustrate a full structure optimization workflow, with methanol as the example.

To minimize computational cost, we perform the final quantum chemical structure optimization at a minimal level of theory (HF with the STO-3G basis set). For practical calculations, DFT or MP2 as well as a larger basis set should be used.

In [11]:
import py3Dmol as p3d
import veloxchem as vlx
import py3Dmol as p3d
import numpy as np

## Initial structure

We obtain the initial (force field optimized) structure from Open Babel, using the SMILES string (`CO`).

In [5]:
methanol_ff = vlx.Molecule.read_smiles('CO')

## xTB optimization

Next, set up the xTB driver and perform the structure optimization.

In [6]:
xtb_drv = vlx.XtbDriver()
xtb_drv.set_method('gfn2')
xtb_grad_drv = vlx.XtbGradientDriver()
xtb_opt_drv = vlx.OptimizationDriver(xtb_grad_drv)
methanol_xtb = xtb_opt_drv.compute(methanol_ff, xtb_drv)

                                                                                                                          
                                                Optimization Driver Setup                                                 
                                                                                                                          
                                     Coordinate System       :    TRIC                                                    
                                     Constraints             :    No                                                      
                                     Max. Number of Steps    :    300                                                     
                                     Transition State        :    No                                                      
                                     Hessian                 :    never                                                   
                

## HF optimization

Set up the SCF and optimization drivers and performing the final optimization, using the xTB results as the initial structure.

In [8]:
basis = vlx.MolecularBasis.read(methanol_xtb, 'STO-3G')
scf_drv = vlx.ScfRestrictedDriver()
scf_results = scf_drv.compute(methanol_xtb, basis)
grad_drv = vlx.ScfGradientDriver()
opt_drv = vlx.OptimizationDriver(grad_drv)
methanol_hf = opt_drv.compute(methanol_xtb, basis, scf_drv)

## Comparison

Visualize the results using py3Dmol:

In [9]:
viewer = p3d.view(viewergrid=(1, 3), width=500, height=200, linked=False)
viewer.addModel(methanol_ff.get_xyz_string(), 'xyz', viewer=(0, 0))
viewer.addModel(methanol_xtb.get_xyz_string(), 'xyz', viewer=(0, 1))
viewer.addModel(methanol_hf.get_xyz_string(), 'xyz', viewer=(0, 2))
viewer.setViewStyle({"style": "outline", "width": 0.05})
viewer.setStyle({"stick":{},"sphere": {"scale":0.25}})
viewer.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

These structures are all very similar, featuring, *e.g*, the staggered H-O-C-H dihedral. To better see any differences in the structures, the distance matrices can be calculated using functionalities from the molecule object:

In [12]:
print('From force field (MMFF94):')
dm_ff = methanol_ff.get_distance_matrix_in_angstrom()
print(np.around(dm_ff,3))
print()

print('From xTB:')
dm_xtb = methanol_xtb.get_distance_matrix_in_angstrom()
print(np.around(dm_xtb,3))
print()

print('From HF:')
dm_hf = methanol_hf.get_distance_matrix_in_angstrom()
print(np.around(dm_hf,3))

From UFF:
[[0.    1.398 1.112 1.112 1.109 1.925]
 [1.398 0.    2.065 2.065 2.05  0.992]
 [1.112 2.065 0.    1.818 1.806 2.291]
 [1.112 2.065 1.818 0.    1.806 2.291]
 [1.109 2.05  1.806 1.806 0.    2.855]
 [1.925 0.992 2.291 2.291 2.855 0.   ]]

From xTB:
[[0.    1.406 1.096 1.096 1.088 1.946]
 [1.406 0.    2.078 2.078 2.008 0.963]
 [1.096 2.078 0.    1.783 1.777 2.343]
 [1.096 2.078 1.783 0.    1.777 2.343]
 [1.088 2.008 1.777 1.777 0.    2.817]
 [1.946 0.963 2.343 2.343 2.817 0.   ]]

From HF:
[[0.    1.433 1.095 1.095 1.091 1.928]
 [1.433 0.    2.109 2.109 2.048 0.991]
 [1.095 2.109 0.    1.773 1.77  2.316]
 [1.095 2.109 1.773 0.    1.77  2.316]
 [1.091 2.048 1.77  1.77  0.    2.831]
 [1.928 0.991 2.316 2.316 2.831 0.   ]]


This gives the the distances between all pairs of atoms, as expressed in Å.

As long as the atom order is consistent (which it here is), we can print the differences in distance matrices, or maximal absolute difference:

In [13]:
print('Difference between HF and xTB:')
print(np.around(dm_hf - dm_xtb,3))
print()
print('Maximal absolute difference between HF and xTB:')
print(f'{np.max(np.abs(dm_hf - dm_xtb)):.4f} Å')

Difference between HF and xTB:
[[ 0.     0.027 -0.    -0.     0.003 -0.019]
 [ 0.027  0.     0.031  0.031  0.04   0.028]
 [-0.     0.031  0.    -0.01  -0.007 -0.027]
 [-0.     0.031 -0.01   0.    -0.007 -0.027]
 [ 0.003  0.04  -0.007 -0.007  0.     0.014]
 [-0.019  0.028 -0.027 -0.027  0.014  0.   ]]

Maximal absolute difference between HF and xTB:
0.0395 Å


The largest difference is thus seen to be 0.04 Å, between atoms 2 and 5. These atoms are the oxygen and one hydrogen from CH$_4$, which has a total distance of 2.05 Å. As any pair-wise differences will propagate along the molecule, it may be more relevant to look at *relative* differences:

In [9]:
# replace the diagonal elements (zeros) with a large number to avoid nan
dm_tmp = dm_hf + 1e10*np.diag(np.ones(len(dm_hf)))

print('Relative difference between HF and xTB:')
print(np.around((dm_hf - dm_xtb)/dm_tmp,3))
print()
print('Maximal relative difference between HF and xTB:')
print(f'{np.max(np.abs((dm_hf - dm_xtb)/dm_tmp)):.4f}')

Relative difference between HF and xTB:
[[ 0.     0.019 -0.    -0.     0.003 -0.01 ]
 [ 0.019  0.     0.015  0.015  0.019  0.029]
 [-0.     0.015  0.    -0.006 -0.004 -0.012]
 [-0.     0.015 -0.006  0.    -0.004 -0.012]
 [ 0.003  0.019 -0.004 -0.004  0.     0.005]
 [-0.01   0.029 -0.012 -0.012  0.005  0.   ]]

Maximal relative difference between HF and xTB:
0.0286


Now we see that the largest *relative* difference in atom-pair distance is between atoms 2 and 6, which correspond to the oxygen and the hydrogen directly connected to the oxygen. With this, we have thus found the largest relative deviation in a bond length, which is in most cases more important than in distances between atoms at different sites.